# Creating our first project

Recipe for

1) creating a new project
2) importing ecoinvent into the new project

This needs to be done once before you can use ecoinvent in brightway. To import ecoinvent, you need to have a local copy of the ecoinvent database. Download it from https://ecoquery.ecoinvent.org by logging in, then choosing "Files" from the top menu. Choose your preferred system model and download the unit processes. Unzip. The path to the "datasets" folder must be specified below.

I recommend choosing a short, descriptive name for the database as you will use it a lot.

In [5]:
# imports and user definitions
import brightway2 as bw

project_name = "new_project"
path_to_datasets = "S:/Benjamin Portner/docs/databases/ecoinvent 3.5 APOS/datasets"
database_name = "ei35"

In [2]:
# create empty project
bw.projects.set_current(project_name)

We created an empty project folder. Next, we will install the biosphere exchanges and the LCIA methods (characterization factors). This could take a while. Wait until you see the line "Creating core data migrations".

In [3]:
# create biosphere and LCIA methods
bw.bw2setup()

Creating default biosphere



Writing activities to SQLite3 database:


Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applied 2 strategies in 0.01 seconds


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 11/18/2019 10:54:52
  Finished: 11/18/2019 10:54:53
  Total time elapsed: 00:00:00
  CPU %: 66.60
  Memory %: 1.81
Created database: biosphere3
Creating default LCIA methods

Applying strategy: normalize_units
Applying strategy: set_biosphere_type
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
Applied 4 strategies in 1.88 seconds
Wrote 850 LCIA methods with 219059 characterization factors
Creating core data migrations



Now that the basic setup is done, we can import the database. This will take a while. Wait until you see the line "Extracted x datasets in y seconds".

In [6]:
# import ecoinvent data
ei35 = bw.SingleOutputEcospold2Importer(path_to_datasets, database_name, use_mp = True)

Extracting XML data from 16045 datasets
Extracted 16045 datasets in 133.29 seconds


By default, there are some anomalies in the ecoinvent datasets that need to be fixed before it can be used in brightway. So-called strategies do exactly that. Each strategy goes through all datasets and manipulates them in a specific way, e.g. drop exchanges with amount zero or delete unlinked exchanges. There is more than one strategy necessary to make ecoinvent work. Luckily, there is a convenience function that applies them all:

In [7]:
# fix some stuff
ei35.apply_strategies()

Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: fix_ecoinvent_flows_pre35
Applying strategy: drop_temporary_outdated_biosphere_flows
Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges
Applying strategy: remove_uncertainty_from_negative_loss_exchanges
Applying strategy: fix_unreasonably_high_lognormal_uncertainties
Applying strategy: set_lognormal_loc_value
Applying strategy: convert_activity_parameters_to_list
App

Let's check if everything worked alright. The output below should indicate "0 unlinked exchanges". The database cannot be saved if there are unlinked exchanges left.

In [8]:
# check if all exchanges are linked
ei35.statistics()

16045 datasets
798743 exchanges
0 unlinked exchanges
  


(16045, 798743, 0)

It worked! Now we can save the database to our hard drive. This may take a while. Wait until you see the line "Brightway2 SQLiteBackend: (name)". 

Once the database is saved to disk, we can always use it in this project. We do **not** need to import it again unless we want to create a new project (and even then, I recommend copying the database from the current project instead of going through the whole import process again).

In [9]:
# if everything worked fine, write database to disk
ei35.write_database()

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:02:20


Title: Writing activities to SQLite3 database:
  Started: 11/18/2019 11:14:16
  Finished: 11/18/2019 11:16:37
  Total time elapsed: 00:02:20
  CPU %: 58.20
  Memory %: 24.72
Created database: ei35


Brightway2 SQLiteBackend: ei35

Done! Now we can use ecoinvent to do our first LCA.